In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataset = pd.read_csv('/content/dataset.csv')

In [ ]:
dataset['target'].unique()

## Cleaning the texts

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []

In [ ]:
for i in range(0, len(dataset)):
  data = re.sub('[^a-zA-Z]', ' ', dataset['data'][i])
  data = data.lower()
  data = data.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  data = [ps.stem(word) for word in data if not word in set(all_stopwords)]
  data = ' '.join(data)
  corpus.append(data)

In [ ]:
print(corpus)

In [ ]:
len(corpus)

## Bag of Words model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

In [ ]:
for train_ix, test_ix in sss.split(X,y):
    X_train = X[train_ix]
    X_test = X[test_ix]
    y_train = y[train_ix]
    y_test = y[test_ix]

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X, y)

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.externals import joblib
pickle_out= open("/content/model.pkl","wb")
joblib.dump(classifier, pickle_out)

In [ ]:
NB_spam_model = open('model.pkl','rb')
clf = joblib.load(NB_spam_model)

In [ ]:
clf.predict(X_test)

In [ ]:
confusion_matrix(y_test, clf.predict(X_test))

In [ ]:
cm